In [8]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
from PIL import Image
import requests
import json
import os

# Load EEG EDF and Convert to Multiple Spectrograms
def edf_to_spectrograms(edf_path, save_dir, window_size=5, overlap=2):
    raw = mne.io.read_raw_edf(edf_path, preload=True)
    sfreq = int(raw.info['sfreq'])  # Sampling frequency
    data, _ = raw[:]
    os.makedirs(save_dir, exist_ok=True)
    spectrogram_paths = []
    
    step_size = window_size * sfreq - overlap * sfreq
    for i in range(0, data.shape[1] - window_size * sfreq, step_size):
        plt.specgram(data[0, i:i + window_size * sfreq], Fs=sfreq, cmap="inferno")
        plt.axis("off")
        img_path = os.path.join(save_dir, f"spectrogram_{i}.png")
        plt.savefig(img_path, bbox_inches="tight", pad_inches=0)
        plt.close()
        spectrogram_paths.append(img_path)
    
    return spectrogram_paths

# Define CNN Model Class
class EEG_CNN(torch.nn.Module):
    def __init__(self, num_classes=5):
        super(EEG_CNN, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = torch.nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = torch.nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.pool = torch.nn.MaxPool2d(2, 2)
        self.fc1 = torch.nn.Linear(256 * 16 * 16, 1024)
        self.fc2 = torch.nn.Linear(1024, 512)
        self.fc3 = torch.nn.Linear(512, num_classes)
        self.dropout = torch.nn.Dropout(0.4)
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Load Trained Model
model = EEG_CNN(num_classes=5)
model.load_state_dict(torch.load("/Users/pavankonam/Downloads/Dataset for EEG/best_spectrogram_model.pth", map_location=torch.device("cpu")))
model.eval()

# Define Image Transformation Pipeline
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Classify Spectrograms and Find Dominant Category
def classify_spectrograms(image_paths):
    categories = ["Adventure & Movement", "Fear & Uncertainty", "People & Social Interaction", "Abstract & Thought-based", "Miscellaneous & Unclear"]
    predictions = []
    
    for image_path in image_paths:
        image = Image.open(image_path).convert("RGB")
        image = transform(image).unsqueeze(0)
        with torch.no_grad():
            output = model(image)
            _, predicted = output.max(1)
        predictions.append(categories[predicted.item()])
    
    return max(set(predictions), key=predictions.count)  # Most common category

# Azure AI Foundry - DALL·E 3 Image Generation
AZURE_ENDPOINT = "https://bol21-m8e8eh4p-eastus.openai.azure.com/"
API_KEY = "2hbkJiNh1QKgNJtJiIEHmLJHAQx1XfrNxAZTxOoiwPHRvWTdvE4DJQQJ99BCACYeBjFXJ3w3AAAAACOGc5AI"

def generate_dream_image(category):
    prompts = {
        "Adventure & Movement": "A surreal painting of a dreamer traveling through mystical landscapes.",
        "Fear & Uncertainty": "A dark, eerie scene with shifting shadows and unknown fears.",
        "People & Social Interaction": "An abstract dream world filled with lively conversations and floating faces.",
        "Abstract & Thought-based": "A surreal visualization of thoughts floating in a cosmic void.",
        "Miscellaneous & Unclear": "A dreamy, undefined scene with strange elements blending together."
    }

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    data = {
        "prompt": prompts[category],
        "n": 1,
        "size": "1024x1024"
    }

    #url = f"{AZURE_ENDPOINT}openai/deployments/dall-e-3/images/generations?api-version=2024-02-01"
    url = f"{AZURE_ENDPOINT}openai/deployments/dall-e-3/images/generations?api-version=2024-02-01"

    response = requests.post(url, headers=headers, json=data)

    # 🔍 Debugging: Print the full API response
    print(f"Response Status Code: {response.status_code}")
    print(f"Response Text: {response.text}")

    try:
        response_json = response.json()
        return response_json["data"][0]["url"]
    except KeyError:
        return f"Error: Unexpected API response format - {response_json}"

# End-to-End Execution
def process_edf_to_art(edf_path):
    save_dir = "/Users/pavankonam/Downloads/Dataset for EEG/test"
    spectrogram_paths = edf_to_spectrograms(edf_path, save_dir)
    
    if not spectrogram_paths:
        print("No spectrograms generated.")
        return
    
    category = classify_spectrograms(spectrogram_paths)
    print(f"🧠 EEG Category: {category}")
    
    image_url = generate_dream_image(category)
    print(f"🎨 Generated Image: {image_url}")

# Run the full process
edf_file_path = "/Users/pavankonam/Downloads/Dataset for EEG/New Database/EEG-waves/subject452_SO7.edf"  # Replace with actual EDF file path
process_edf_to_art(edf_file_path)


Extracting EDF parameters from /Users/pavankonam/Downloads/Dataset for EEG/New Database/EEG-waves/subject452_SO7.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 29219  =      0.000 ...    73.047 secs...
🧠 EEG Category: Adventure & Movement
Response Status Code: 200
Response Text: {"created":1742320207,"data":[{"content_filter_results":{"hate":{"filtered":false,"severity":"safe"},"self_harm":{"filtered":false,"severity":"safe"},"sexual":{"filtered":false,"severity":"safe"},"violence":{"filtered":false,"severity":"safe"}},"prompt_filter_results":{"hate":{"filtered":false,"severity":"safe"},"profanity":{"detected":false,"filtered":false},"self_harm":{"filtered":false,"severity":"safe"},"sexual":{"filtered":false,"severity":"safe"},"violence":{"filtered":false,"severity":"safe"}},"revised_prompt":"A surreal painting depicting a dreamer journeying through enchanted and mystical terrains.","url":"https://dalleproduse.blob.core.windows.n